# Common code for NIPS figures

In [1]:
%matplotlib inline
from __future__ import division

import likelihoodfree.io as io
import likelihoodfree.viz as viz
import likelihoodfree.PDF as lfpdf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.signal as ss
import socket
import svgutils.transform as sg

from IPython.display import SVG, display
from mpl_toolkits.axes_grid1 import make_axes_locatable
from math import factorial

In [2]:
def cm2px(cm):
    return int(np.round(cm / 2.54*300))

def svg(img):
    display(SVG(img))

In [8]:
FIG_WIDTH = 16
FIG_WIDTH_CM = str(FIG_WIDTH) + 'cm'
FIG_WIDTH_PX = cm2px(FIG_WIDTH)

FONT_SIZE_PANEL = 20

In [ ]:
HOSTNAME = socket.gethostname()

if HOSTNAME == 'nsa3004':
    PATH_DROPBOX = '/home/jm/Mackelab/team/Write/Manuscripts/2017_NIPS_NeuralModelInference/'
else:
    raise ValueError('Unknown hostname {}, add in if-else block'.format(HOSTNAME))

PATH_DROPBOX_FIGS = PATH_DROPBOX + 'figs/'